# Proof of concept of Imviz requirements using glupyter/bqplot

We start off by silencing warnings that can happen when loading data as well as deprecation warnings, for clarity:

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
from astropy.utils.data import download_file

gc_2mass_j = download_file('https://www.astropy.org/astropy-data/galactic_center/gc_2mass_j.fits', cache=True)
gc_msx_e = download_file('https://www.astropy.org/astropy-data/galactic_center/gc_msx_e.fits', cache=True)

We start off by looking at some of the basic features:

In [ ]:
import matplotlib.pyplot as plt
from glue.plugins.wcs_autolinking.wcs_autolinking import wcs_autolink, WCSLink

from jdaviz import Imviz

imviz = Imviz()
imviz.load_data(gc_2mass_j, data_label='gc_2mass_j')
imviz.load_data(gc_msx_e, data_label='gc_msx_e')

viewer = imviz.app.get_viewer('viewer-1')

# Manually link the data. We can remove this when Imviz auto-linking issue is resolved.
# This is necessary for blink to function properly.
wcs_links = wcs_autolink(viewer.session.data_collection)
for link in wcs_links:
    exists = False
    for existing_link in viewer.session.data_collection.external_links:
        if isinstance(existing_link, WCSLink):
            if (link.data1 is existing_link.data1
                    and link.data2 is existing_link.data2):
                exists = True
                break
    # Add only those links that don't already exist
    if not exists:
        viewer.session.data_collection.add_link(link)

# Because linking happens after load, the image display is broken a little.
# So, please do this manually **after** running this cell.
#
# 1. Uncheck both data from Data menu.
# 2. Re-check both data from Data menu.

imviz.app

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
viewer.state.layers[0].stretch = 'sqrt'

the colormap:

In [ ]:
viewer.state.layers[0].cmap = plt.cm.viridis

the limits via the percentile option:

In [ ]:
viewer.state.layers[0].percentile = 99

or the limits directly:

In [ ]:
viewer.state.layers[0].v_min = 150
viewer.state.layers[0].v_max = 1000

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to astropy regions. Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.app.get_subsets_from_viewer('viewer-1')

In [ ]:
regions

Since the region is an astropy region, we can e.g. convert it to a mask:

In [ ]:
mask = regions['Subset 1'].to_mask(mode='subpixels')

In [ ]:
data = imviz.app.get_data_from_viewer('viewer-1', 'gc_2mass_j')

In [ ]:
plt.imshow(mask.to_image(data.shape), origin='lower')